## Read earnings reports (OO version)

E. Quinn 12/22/2019

This notebook uses pdfminer to extract the information from the individual earnings report

The documentation for pdfminer is at:

https://buildmedia.readthedocs.org/media/pdf/pdfminer-docs/latest/pdfminer-docs.pdf

Maintenance:

* 3/6/2020  
  * Add check date and number
* 3/7/2020  
  * Align personnel classes with support professionals structure
  * Implement salary step capture for support professionals

## Import standard python datascience packages

In [ ]:
import math
import re
import copy
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid

## Import pdfminer packages

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal

### Show the directory we are running in

In [ ]:
!pwd

## Read the pdf and create a dictionary with the contents of each text box

### Function read_pdf() reads a PDF and returns a dictionary containing the contents

Strategy for this document:  

Save information from each element in the LTTextBox objects in a dictionary including:

- x0 horizontal coordinate of the upper left corner of the text box
- x1 horizontal coordinate of the lower right corner of the text box
- y0 vertical coordinate of the upper left corner of the text box
- y1 vertical coordinate of the lower right corner of the text box
- page number 
- sequence number of text box within this page
- text contained in the text box, converted to ascii

Parsing the text is complicated by the fact that that a text box may span multiple columns and/or rows, and the text box groupings vary quite a bit depending on the page contents and layout.

However, with a bit of luck the structure of the document will allow the contents to be deciphered with the following heuristics:

- Text boxes containing left justified columns will tend to have nearly the same x0 coordinates
- Text boxes containing right justified columns will tend to have nearly the same x1 coordinates
- The codes for fund, account code, and object code are numeric and have fixed lengths
- Extraneous information is often preceded or followed by a series of underscore and newline characters
- Last name can be distinguished because is the only field that is all characters followed by a comma
- Last name may be preceded by between one and three numerical fields:  fund, account, object.  If it is, the x0 value is shifted to the left.
    - Three numerical fields precede the name:  assume they are fund, account, object
    - Two numerical fields precede the name: assume they are account, object
    - One numerical field precedes the name: assume it is object
    

In [ ]:
def read_pdf(path):
    document = open(path, 'rb')                                     #read a pdf and create a document object
    rsrcmgr = PDFResourceManager()                                  #create a resource manager
    laparams = LAParams()                                           #set the parameters for analysis
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)          #create a PDF page aggregator object
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    pdf={}                                                          #dictionary to hold the results

    pageno = -1                                                     #initialize page coounter to zero

    for page in PDFPage.get_pages(document):                        #loop through the pdf page by page
        pageno = pageno + 1                                         #increment the page number
        pdf[pageno] = {}                                            #dictionary for this page
        interpreter.process_page(page)                              # receive the LTPage object for the page.
        layout = device.get_result()                                # create layout object
        tbox_no=0                                                   # index for element number
        for element in layout:
            if (type(element).__name__=='LTTextBoxHorizontal'):     #loop through text boxes
                tbox_no += 1                                        #increment text box number
                pdf[pageno][tbox_no] = {}                           #dictionary for text boxes within page
                x0 = round(element.x0,2)                            #x0 coordinate of textbox corner
                x1 = round(element.x1,2)                            #x1 coordinate of textbox corner
                y0 = round(element.y0,2)                            #y0 coordinate of textbox corner
                y1 = round(element.y1,2)                            #y1 coordinate of textbox corner
                txt = element.get_text().encode('ascii', 'ignore')  #text converted to ascii
                pdf[pageno][tbox_no]['x0'] = x0                     #create x0 coordinate entry
                pdf[pageno][tbox_no]['x1'] = x1                     #create x1 coordinate entry
                pdf[pageno][tbox_no]['y0'] = y0                     #create y0 coordinate entry
                pdf[pageno][tbox_no]['y1'] = y1                     #create y1 coordinate entry

                pdf[pageno][tbox_no]['text'] = ''.join(chr(c) for c in txt) #convert bytes to string
    return(pdf)

### Utility functions

In [ ]:
#remove the commas from earnings and rate values

def remove_commas(st):
    newstr = st.replace(',','')                     #remove commas from string
    return(newstr)

In [ ]:
#remove the headings fields 

def remove_headings(st):
    lines = st.split('\n')                         #split the string at newline characters '\n'
    for line in lines:                             #loop through the resulting lines
        if (line.startswith('FUND ') |\
           (line.startswith('POSITION')) |\
           (line.startswith('RATE')) |\
           (line.startswith('ACCT-')) |\
           (line.startswith('CHECK')) |\
           (line.startswith('_'))):                #check for strings that appear only in headings
            try:
                newline_index = st.index('\n')     #if present, remove this line from the text string
                st = st[newline_index+1:]
            except ValueError:
                print('Value Error',st)            #recover from Value Error and print string
        else:
            return(st)                             #if no headings, just return
    return('')

### Build a dictionary with only those text boxes containing names

Use the following algorithm to identify text boxes that contain names:

- x0, horizontal coordinate of the upper left corner of the text box, is less than 162
- the text string contains at least one comma

In [ ]:
def get_names(dct):

    dnames = {}

    fund = ''
    acct = ''
    obj  = ''
    
    for page in sorted(dct.keys()):                                #loop through text box dictionary by page # 
        if (page not in dnames.keys()):                            #page number is highest level key
            dnames[page] = {}                                      #initialize entry for this page
        for tb in sorted(dct[page].keys()):                        #loop through all text boxes on this page
            if (dct[page][tb]['x0'] < 162.0):                      #those with names start to the left of x0=162
                txt = str(dct[page][tb]['text'])                   #convert the 'text' element to a string
                if (',' in txt):                                   #every name contains a comma
                    txt = remove_headings(txt)
                    lines = txt.split('\n')                        #split text into lines
                    words = lines[0].split()                       #split first line into words
                    for word in words:                             #loop through and strip out fund, acct, obj
                        if (word.isdigit()):
                            if (len(word)==4):                     # 4 digits means fund
                                fund = word
                            if (len(word)==8):                     # 8 digits means acct-code
                                acct = word
                            if (len(word)==5):                     # 5 digits means obj
                                obj = word
                            txt = txt[len(word)+1:]                # remove fund/acct/obj from txt
                    dnames[page][tb] = {}                          #initialize dictionary for this page
                    dnames[page][tb]['x0'] = dct[page][tb]['x0']
                    dnames[page][tb]['x1'] = dct[page][tb]['x1']
                    dnames[page][tb]['y0'] = dct[page][tb]['y0']
                    dnames[page][tb]['y1'] = dct[page][tb]['y1']
                    dnames[page][tb]['fund'] = fund
                    dnames[page][tb]['acct'] = acct
                    dnames[page][tb]['obj'] = obj
                    dnames[page][tb]['text'] = txt
    return(dnames)

In [ ]:
def get_check_numbers(txt):
    newtxt = ''
    for word in txt.split('\n'):
        if (word.isdigit()):
            newtxt = newtxt + word + '\n'
    return(newtxt)

In [ ]:
def get_check_dates(txt):
    newtxt = ''
    for word in txt.split('\n'):
        if (not word.isdigit()):
            newtxt = newtxt + word + '\n'
    return(newtxt)

### Match consolidated text boxes with names to text boxes for other columns

In [ ]:
def get_matches(names,pdf):

    for page in sorted(names.keys()):                                       #loop through pages
        for tb in sorted(names[page].keys()):                               #loop through text boxes
            y0 = names[page][tb]['y0']                                      #extract vertical coordinates
            y1 = names[page][tb]['y1']
            txt = names[page][tb]['text']                                   #extract text
           
            for tb2 in sorted(pdf[page].keys()):                            #loop through the other boxes in pdf
                if (tb != tb2):                                             #ignore if same box as names
                    tx0 = pdf[page][tb2]['x0']                              #get horizontal offset
                    ty0 = pdf[page][tb2]['y0']                              #check whether the vertical 
                    ty1 = pdf[page][tb2]['y1']                              #range of this box overlaps that
                    if ((y0 <= ty1) & (ty0 <= y1)):                         #of the name box
                        txt = remove_headings(pdf[page][tb2]['text'])
                        if ((312.0 < tx0) & (tx0 < 316.0)):                 #match to DATE
                            #names[page][tb]['dates'] = txt
                            names[page][tb]['numbers'] = get_check_numbers(txt)
                            names[page][tb]['dates'] = get_check_dates(txt)
                        if ((437.0 < tx0) & (tx0 < 440.0)):                 #match to POSITION
                            names[page][tb]['positions'] = txt
                        if ((509.0 < tx0) & (tx0 < 533.0)):                 #match to RATE 
                            names[page][tb]['rates'] = remove_commas(txt)
                        if ((558.0 < tx0) & (tx0 < 630.0)):                 #match to ACCT-EARNINGS
                            names[page][tb]['earnings'] = remove_commas(txt)
    return                                                              

### Get sum of earnings lines

Used for balancing to total on earnings report

In [ ]:
def balancing_totals(dct):
    
    total=0.0                                                       #initialize grand totals

    for page in dct.keys():                                         #loop through pages of earnings report
        for tb in dct[page].keys():                                 #loop through the text boxes on this page
            names    = dct[page][tb]['text'].split('\n')            #split the names string into lines
            earnings = dct[page][tb]['earnings'].split('\n')        #split the earnings string into lines
            
            for i in np.arange(len(names)):                         #loop through names
                if (len(names[i]) > 1):                             #
                    earnamt = float(earnings[i])                    #convert the corresponding earnings to float
                    total += earnamt                                #add to running total
                
    print(round(total,2))                                           #print the grand total

    return

### Consolidate text boxes that overlap on the vertical scale and contain names

In [ ]:
def consolidate_name_boxes(names):
    newnames = {}
    
    for page in sorted(names.keys()):                                        #loop through pages of pdf
        newnames[page] = {}                                                  #initialize new names dictionary
        skip = make_array()                                                  #initialize list of boxes to skip
    
        for tb in sorted(names[page].keys()):                                #loop through text boxes on this page
            for tb2 in sorted(names[page].keys()):                           #compare this one to the others
                if ((tb2 > tb) & \
                    (names[page][tb]['y0'] <= names[page][tb2]['y1']) & \
                    (names[page][tb2]['y0'] <= names[page][tb]['y1'])):      
                    d = {}                                                   #initialize replacement entry
                    d['x0'] = names[page][tb]['x0']                          #keep x0    
                    d['x1'] = names[page][tb2]['x1']                         #replace x1 with tb2 value
                    d['y0'] = names[page][tb2]['y0']                         #replace y0 with tb2 value
                    d['y1'] = names[page][tb]['y1']                          #keep y1 value
                    d['text'] = names[page][tb]['text'] +\
                        names[page][tb2]['text']                             #contatenate text strings
                    d['fund'] = names[page][tb]['fund']                      #copy fund, acct, and obj
                    d['acct'] = names[page][tb]['acct']
                    d['obj'] = names[page][tb]['obj']
                    newnames[page][tb2] = d                                  #plug into dictionary
                    skip = np.append(skip,tb)                                #add old boxes to skip list
                    skip = np.append(skip,tb2)
            if (tb not in skip):                                             #if no match, check skip list 
                newnames[page][tb] = names[page][tb]                         #just copy if not in skip list
                    
    return(newnames)

### Read the FY2017 earnings report

In [ ]:
#p17 = read_pdf('../FY17 Gene_Redacted.pdf')
p17 = read_pdf('../FY17 Gene.pdf')

### Read the FY2018 earnings report

In [ ]:
#p18 = read_pdf('../FY18 Gene_Redacted.pdf')
p18 = read_pdf('../FY18 Gene.pdf')

### Process the FY17 Earnings report

In [ ]:
dnames17 = get_names(p17)                                   #extract name records from pdf
consolidated_names17 = consolidate_name_boxes(dnames17)     #consolidate overlapping text boxes containing names
get_matches(consolidated_names17,p17)                       #match text boxes for other columns by vertical pos

### Process the FY18 Earnings report

In [ ]:
dnames18 = get_names(p18)                                   #extract name records from pdf
consolidated_names18 = consolidate_name_boxes(dnames18)     #consolidate overlapping text boxes containing names
get_matches(consolidated_names18,p18)                       #match text boxes for other columns by vertical pos

### Check sum of earnings against total on earnings reports

In [ ]:
balancing_totals(consolidated_names17)     #FY2017 earnings report total is $22,608,024.34

In [ ]:
balancing_totals(consolidated_names18)     #FY2018 earnings report total is $22,409,915.41

In [ ]:
class EG_acct_codes():
    def __init__(self):
        self.EG_account_codes ={
            '71100105': 'K Frenchtown',
            '71100107': 'K MDBK',
            '71109105': 'Title 1  Frenchtown',
            '71109107': 'Title 1  MDBK',
            '71110105': 'Grade 1 Frenchtown',
            '71110107': 'Grade 1 MDBK',
            '71120105': 'Grade 2 Frenchtown',
            '71120107': 'Grade 2 MDBK',
            '71121102': 'Art Eldredge',
            '71121103': 'Art Cole',
            '71121105': 'Art Frenchtown',
            '71121106': 'Art EGHS',
            '71121107': 'Art MDBK',
            '71121108': 'Art Hanaford',
            '71123103': 'ELA Cole',
            '71123106': 'ELA EGHS',
            '71123108': 'ELA Hanaford',
            '71124103': 'Foreign Language Cole',
            '71124106': 'Foreign Language EGHS',
            '71125102': 'PE/health Eldredge',
            '71125103': 'PE/health Cole',
            '71125105': 'PE/health Frenchtown',
            '71125106': 'PE/health EGHS',
            '71125107': 'PE/health MDBK',
            '71125108': 'PE/health Hanaford',
            '71126103': 'Tech Cole',
            '71126306': 'Tech EGHS',
            '71127103': 'Math Cole',
            '71127106': 'Math EGHS',
            '71128102': 'Music Eldredge',
            '71128103': 'Music Cole',
            '71128105': 'Music Frenchtown',
            '71128106': 'Music EGHS',
            '71128107': 'Music MDBK',
            '71128108': 'Music Hanaford',
            '71129103': 'Science Cole',
            '71129106': 'Science EGHS',
            '71130102': 'Grade 3 Eldredge',
            '71130105': 'Grade 3 Frenchtown',
            '71130108': 'Grade 3 Hanaford',
            '71130406': 'Business/Computer  EGHS',
            '71131103': 'Social Studies Cole',
            '71131106': 'Social Studies EGHS',
            '71140102': 'Grade 4 Eldredge',
            '71140108': 'Grade 4 Hanaford',
            '71140403': 'Computer Cole',
            '71140406': 'Computer EGHS',
            '71141102': 'Reading Eldredge',
            '71141103': 'Reading Cole',
            '71141105': 'Reading Frenchtown',
            '71141106': 'Reading EGHS',
            '71141107': 'Reading MDBK',
            '71141108': 'Reading Hanaford',
            '71150102': 'Grade 5 Eldredge',
            '71150108': 'Grade 5 Hanaford',
            '71180102': 'SPED Eldredge',
            '71180103': 'SPED EGHS',
            '71180105': 'SPED Frenchtown',
            '71180106': 'SPED EGHS',
            '71180107': 'SPED MDBK',
            '71180108': 'SPED Hanaford',
            '71181102': 'SPED Life Skills Eldredge',
            '71181103': 'SPED Life Skills Cole',
            '71181105': 'SPED Life Skills Frenchtown',
            '71181106': 'SPED Life Skills EGHS',
            '71181107': 'SPED Life Skills MDBK',
            '71182107': 'SPED EGHS',
            '71191302': 'ESL Eldredge',
            '71191303': 'ESL Cole',
            '71191305': 'ESL Frenchtown',
            '71191306': 'ESL EGHS',
            '71191307': 'ESL MDBK',
            '71191308': 'ESL Hanaford',
            '71210202': 'Teacher Subs Eldredge',
            '71210203': 'Teacher Subs Cole',
            '71210205': 'Teacher Subs Frenchtown',
            '71210206': 'Teacher Subs EGHS',
            '71210207': 'Teacher Subs MDBK',
            '71210208': 'Teacher Subs Hanaford',
            '71210402': 'Long Term Subs Eldredge',
            '71210403': 'Long Term Subs Cole',
            '71210405': 'Long Term Subs Frenchtown',
            '71210406': 'Long Term Subs EGHS',
            '71210407': 'Long Term Subs MDBK',
            '71210408': 'Long Term Subs Hanaford',
            '71223102': 'Para Subs Eldredge',
            '71223103': 'Para Subs Cole',
            '71223105': 'Para Subs Frenchtown',
            '71223106': 'Para Subs EGHS',
            '71223107': 'Para Subs MDBK',
            '71223108': 'Para Subs Hanaford',
            '71231503': 'Guidance Cole',
            '71231506': 'Guidance EGHS',
            '71246702': 'Librarian Eldredge',
            '71246703': 'Librarian Cole',
            '71246705': 'Librarian Frenchtown',
            '71246706': 'Librarian EGHS',
            '71246707': 'Librarian MDBK',
            '71246708': 'Librarian Hanaford',
            '71269506': 'Nurse EGHS',
            '71270102': 'Nurse Subs Eldredge',
            '71270103': 'Nurse Subs Cole',
            '71270105': 'Nurse Subs Frenchtown',
            '71270106': 'Nurse Subs EGHS',
            '71270107': 'Nurse Subs MDBK',
            '71270108': 'Nurse Subs Hanaford',
            '71270302': 'Nurse Eldredge',
            '71270303': 'Nurse Cole',
            '71270305': 'Nurse Frenchtown',
            '71270306': 'Nurse EGHS',
            '71270307': 'Nurse MDBK',
            '71270308': 'Nurse Hanaford',
            '71301106': 'SPED EGHS',
            '71301602': 'Social Worker Eldredge',
            '71301603': 'Social Worker Cole',
            '71301605': 'Social Worker Frenchtown',
            '71301606': 'Social Worker EGHS',
            '71301607': 'Social Worker MDBK',
            '71301608': 'Social Worker Hanaford',
            '71302702': 'OT Eldredge',
            '71302703': 'OT Cole',
            '71302705': 'OT Frenchtown',
            '71302706': 'OT EGHS',
            '71302707': 'OT MDBK',
            '71302708': 'OT Hanaford',
            '71308102': 'Adaptive PE Eldredge',
            '71308103': 'Adaptive PE Cole',
            '71308105': 'Adaptive PE Frenchtown',
            '71308106': 'Adaptive PE EGHS',
            '71308107': 'Adaptive PE MDBK',
            '71308108': 'Adaptive PE Hanaford',
            '71310106': 'History EGHS',
            '71311702': 'Psychologist Eldredge',
            '71311703': 'Psychologist Cole',
            '71311705': 'Psychologist Frenchtown',
            '71311706': 'Psychologist EGHS',
            '71311707': 'Psycholotist MDBK',
            '71311708': 'Psychologist Hanaford',
            '71321802': 'Speech Eldredge',
            '71321803': 'Speech Cole',
            '71321805': 'Speech Frenchtown',
            '71321806': 'Speech EGHS',
            '71321807': 'Speech MDBK',
            '71321808': 'Speech Hanaford',
            '71347302': 'Custodian Subs Eldredge',
            '71347303': 'Custodian Subs Cole',
            '71347305': 'Custodian Subs Frenchtown',
            '71347306': 'Custodian Subs EGHS',
            '71347307': 'Custodian Subs MDBK',
            '71347308': 'Custodian Subs Hanaford'
        }
        self.local_to_ucoa = {
            '711001': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 1,'JC':1100,'Sub Desc':'K'},
            '711101': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 3,'JC':1100, 'Sub Desc': 'Grade 1'},
            '711201': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 4,'JC':1100, 'Sub Desc': 'Grade 2'},
            '711301': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 5,'JC':1100, 'Sub Desc': 'Grade 3'},
            '711401': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 6,'JC':1100, 'Sub Desc': 'Grade 4'},
            '711501': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 7,'JC':1100, 'Sub Desc': 'Grade 5'},
            '711211': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 200,'JC':1100, 'Sub Desc': 'Art'},
            '711231': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 500,'JC':1100, 'Sub Desc': 'ELA'},
            '711241': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 700,'JC':1100, 'Sub Desc': 'Foreign Language'},
            '711251': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 1200,'JC':1100, 'Sub Desc': 'PE/health'},
            '711271': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 1500,'JC':1100, 'Sub Desc': 'Math'},
            '711281': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 1600,'JC':1100, 'Sub Desc': 'Music'},
            '711291': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 1700,'JC':1100, 'Sub Desc': 'Science'},
            '711311': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 1900,'JC':1100, 'Sub Desc': 'Social Studies'},
            '711411': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 2400,'JC':1100, 'Sub Desc': 'Reading'},
            '711261': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 2000,'JC':1100, 'Sub Desc': 'Tech/computer'},
            '711404': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 2000,'JC':1100, 'Sub Desc': 'Tech/computer'},
            '712467': {'Fund':1000000,'Prog':10,'Func':212,'Sub': 2600,'JC':1600, 'Sub Desc': 'Librarian'},
            '713027': {'Fund':1000000,'Prog':20,'Func':232,'Sub': 2125,'JC':1700, 'Sub Desc': 'Occupational Therapy'},
            '713218': {'Fund':1000000,'Prog':20,'Func':232,'Sub': 2122,'JC':1700, 'Sub Desc': 'Speech Therapy'},
            '713117': {'Fund':1000000,'Prog':20,'Func':232,'Sub': 2121,'JC':1700, 'Sub Desc': 'Psychologist'},
            '713016': {'Fund':1000000,'Prog':20,'Func':232,'Sub': 2120,'JC':1700, 'Sub Desc': 'Social Worker'},
            '719130': {'Fund':1000000,'Prog':40,'Func':111,'Sub': 600,'JC':1300, 'Sub Desc': 'ESL'},
            '711304': {'Fund':1000000,'Prog':10,'Func':111,'Sub': 1800,'JC':1100, 'Sub Desc': 'Business/comp'},
            '712315': {'Fund':1000000,'Prog':10,'Func':211,'Sub': 800,'JC':1500, 'Sub Desc': 'Guidance'}
        }
        return
        
    def get_eg_acct_desc(self,acct):
        """Provides descriptions for accounting codes in EG MUNIS system."""
        try:
            return(self.EG_account_codes[acct])
        except KeyError:
            return('(no description)')
    

## Define classes and functions

Object oriented classes for processing earnings

In [ ]:
class Person():                                                         #generic employee class
    def __init__(self,name,uuid):                                       #constructor
        self.name = name
        self.uuid = uuid
        self.start_date = None
        self.retirement_date = None
        self.resignation_date = None
        self.payperiods = {}
        self.textbox = {}
        return
        
    def add_payperiod(self,date):                                       #add a payperiod
        if (date not in self.payperiods.keys()):
            self.payperiods[date] = {}
        return
    
    def add_textbox(self,fyear,page,tbno,seq,tboxinfo):                 #add a textbox from earnings report
        if (fyear not in self.textbox.keys()):
            self.textbox[fyear] = {}
        if (page not in self.textbox[fyear].keys()):
            self.textbox[fyear][page] = {}
        if (tbno not in self.textbox[fyear][page].keys()):
            self.textbox[fyear][page][tbno] = {}
        self.textbox[fyear][page][tbno][seq] = tboxinfo
        
        return
    
    def get_textboxes(self):                                            #return textbox dictionary
        return(self.textbox)
    
    def get_name(self):                                                 #return name of person
        return(self.name)
    
    def get_uuid(self):                                                 #return uuid for this person
        return(self.uuid)
    
class Teacher(Person):                                                  #Teacher - inherits from Person
    
    def __init__(self,name,uuid):                                       #constructor
        super().__init__(name,uuid)                                     #call constructor of superclass
        
        self.cba_cols ={'B': 0,'B+30': 1,'M': 2,'M+30': 3,'M2/CAGS': 4, 'D': 5}
        
        self.cba = np.zeros((10, 10, 6))    #salary matrix is 3-D numpy array indexed by: fyear, step, column
        
                                                                #start with FY2016 (2015-2016) salary matrix
        self.cba[3,:,:] = np.array([
            [41286, 42900, 43871, 44505, 44893, 45186],
            [44871, 46484, 47454, 48085, 48474, 48771],
            [48494, 50106, 51078, 51709, 52098, 52393],
            [52118, 53729, 54700, 55332, 55722, 56018],
            [55743, 57354, 58328, 58958, 59347, 59642],
            [59366, 60979, 61951, 62583, 62974, 63266],
            [62991, 64605, 65574, 66206, 66596, 66892],
            [66616, 68228, 69199, 69829, 69806, 70515],
            [71741, 73353, 74323, 74954, 75345, 75639],
            [78898, 80675, 81743, 82438, 82866, 83190]]) 
        
                                                                #FY2017 is the same as FY2016
        self.cba[4,:,:] = self.cba[3,:,:]
        
                                                                #FY2018 2% increase
        self.cba[5,:,:] = np.around(1.02*self.cba[4,:,:],0)
        
                                                                #FY2019 2.25% increase
        self.cba[6,:,:] = np.around(1.0225*self.cba[5,:,:],0)
        
                                                                #FY2020 same as FY2019
        self.cba[7,:,:] = self.cba[6,:,:]
        
                                                                #FY2021 2% increase
        self.cba[8,:,:] = np.around(1.02*self.cba[7,:,:],0)
        
                                                                #FY2022 2.25% increase
        self.cba[9,:,:] = np.around(1.0225*self.cba[8,:,:],0)

        
                                                                #FY2015: back out 2% increase from FY2016
        self.cba[2,:,:] = np.around(self.cba[3,:,:]/1.02,0) 
        
                                                                #FY2014: back out 2.5% increase from FY2015
        self.cba[1,:,:] = np.around(self.cba[2,:,:]/1.025,0)  
        
                                                                #FY2013: back out 1.01% from FY2014 for steps 1-9
        self.cba[0,0:8,:] = np.around(self.cba[1,0:8,:]/1.01,0)
                                                                #FY2013: back out 2.25% from FY2014 for step 10
        self.cba[0,9,:]   = np.around(self.cba[1,9,:]/1.0225,0)  
        return                                                      
        
    def add_payperiod(self,date):                                      #defined in superclass????                                 
        return(super().add_payperiod(date))
    
    def add_textbox(self,fyear,page,tbno,seq,tbox):
        super().add_textbox(fyear,page,tbno,seq,tbox)
        rate = tbox['rate']
        fyear = tbox['fyear']
        earnings = tbox['earnings']
        tbx = self.textbox[fyear][page][tbno][seq]
        self.decode_earnings(fyear,rate,earnings,tbx)
        return
    
    def get_textboxes(self):
        return(super().get_textboxes())
        
    def get_name(self):
        return(super().get_name())
    
    def get_uuid(self):
        return(super().get_uuid())
    
    def get_salary(self,fyear,step,col):                        #look up salary by year, column, step
        """Returns CBA salary given fiscal year, column, and step for FY2013-FY2022."""
        yr = fyear - 2013                                       #year index 0 is 2013
        s  = step-1                                             #step index is one less than the step number
        c  = col                                                #column within the CBA salary matrix
        try:
            return self.cba[yr,s,c]                             #return the value if it exists
        except KeyError:                                        #otherwise raise error condition
            print("KeyError in get_salary: ",yr,s,c)
        except IndexError:
            print("IndexError in get_salary: ",yr,s,c)
    
    def decode_earnings(self,fyear,rate,earnings,dct):  #compute step, FTE, # of payments for teachers
        
        f = [1.0,1/2.,1/10.,1/5.,4/5.,3/10.,\
            4/10.,6/10.,1/20.,1/3.,2/3.,1/4.,\
            3/4.,1/5.,2/5.,3/5.,4/5.,1/6.,5/6.,\
            1/7.,2/7.,3/7.,4/7.,5/7.,6/7.,1/8.,\
            3/8.,5/8.,7/8.,1/9.,2/9.,4/9.,5/9.,\
             7/9.,8/9.,7/10.,9/10.]                    #possible FTE fractions
        
        cbacols ={'B': 0,'B+30': 1,'M': 2,\
                  'M+30': 3,'M2/CAGS': 4, 'D': 5}               #cba salary matrix columns

        salary = round(184.0*rate,0)                            #salary is 184 times daily rate
        lower_bound = salary - 1.0                              #lower limit for tolerance
        upper_bound = salary + 1.0                              #upper limit for tolerance
        step_code = ''                                          #initialize step code
        n_payments = [26.0,21.0]                                #possible number of payments: 26 or 21                              
        fte = np.NaN
        payments = np.NaN
        min_abs_diff = 10000.     
                                                                #search salary matrix for a match
        for y in [fyear-1,fyear]:                               #check this year and one year before
            for step in np.arange(1,11):                        #loop through steps
                for col in self.cba_cols.keys():                #loop through columns
                    cbasal = self.get_salary(y,\
                        step,self.cba_cols[col])                #salary from CBA matrix
                    if( (lower_bound <= cbasal) &\
                       (cbasal <= upper_bound) ):               #computed salary within $1 of CBA
                        step_code = str(y) + '-' + col + '-' + str(step)    #code is:  yyyy-cat-step

        if ((rate > 200.) | (earnings > 1000.)):       #determine fte and payments from rate
            for p in n_payments:                                #loop through payments 26 and 21
                for frac in f:                                  #loop through the FTE fractions in the list
                    diff = abs(earnings - 184.0*rate*frac/p)    #compute the difference from earnings
                    if (diff < min_abs_diff):                   #see if this is the smallest so far
                        min_abs_diff = diff                     #if it is, then save the min difference
                        payments = p                            #save the number of payments
                        fte = frac                              #save the FTE fraction
        elif ((rate > 0.0) & \
              (rate < 100.) & \
              (earnings > 500.)):                               #do this when rate is too low to be a daily rate
            for p in n_payments:                                #loop through possible payments 26 and 21
                for frac in f:                                  #loop through possible FTE fractions
                    for y in np.arange(fyear-1,fyear+1):        #try this fiscal year and the previous   
                        for s in np.arange(10):                 #loop through CBA salary steps
                            for c in cbacols.keys():            #loop through salary matrix columns
                                sal = self.get_salary(y,s+1,cbacols[c])    #look up salary in CBA tables
                                try:
                                    diff = abs(earnings - frac*sal/p)      #compute delta from earnings
                                except TypeError:
                                    diff = 100000.0
                                if (diff < min_abs_diff):       #if this is the smallest difference so far:
                                    min_abs_diff = diff         #save the smallest value
                                    fte = frac                  #save the FTE fraction
                                    mc = c                      #save the salary matrix column
                                    yr = y                      #save the year used for the CBA salary
                                    salary = sal
                                    payments = p                #save the number of payments
                                    step_code = str(yr) + '-' + mc + '-' + str(s+1)  #construct the step code
        dct['step'] = step_code                                 #save results in Teacher object textbox
        dct['payments'] = payments
        dct['fte'] = fte
        dct['mindiff'] = round(min_abs_diff,4)
        dct['salary'] = round(salary,0)

        return
    
class Teacher_Substitute(Person):
    
    def __init__(self,name,uuid):
        super().__init__(name,uuid)
        return
        
    def add_payperiod(self,date):
        return(super().add_payperiod(date))
    
    def add_textbox(self,fyear,page,tbno,seq,tbox):
        return(super().add_textbox(fyear,page,tbno,seq,tbox))
    
    def get_textboxes(self):
        return(super().get_textboxes())
        
    def get_name(self):
        return(super().get_name())
    
    def get_uuid(self):
        return(super().get_uuid())
    
class Paraprofessional(Person):
    
    def __init__(self,name,uuid):
        super().__init__(name,uuid)
        self.cba = np.zeros((10, 7))    #salary matrix is 2-D numpy array 
                                        #indexed by: fyear, step#start with FY2016 (2016-2017) salary matrix

        self.cba[3,:] = np.array([[15.35, 15.84, 16.19, 16.69, 17.17, 18.06, 19.14]])   #FY2017
        
        self.cba[2,:] = np.around(self.cba[3,:]/1.02,2)
        self.cba[4,:] = self.cba[3,:]                                                   #FY2018  no change
        self.cba[5,:] = np.around(1.02*self.cba[4,:],2)                                 #FY2019  2% increase
        self.cba[6,:] = np.around(1.02*self.cba[5,:],2)                                 #FY2020  2% increase
        self.cba[7,:] = self.cba[6,:]                                                   #FY2021  no change?
        self.cba[8,:] = self.cba[7,:]                                                   #FY2022  no change?
        self.cba[9,:] = self.cba[8,:]                                                   #FY2023  no change?
        return
        
    def add_payperiod(self,date):
        return(super().add_payperiod(date))
    
    def add_textbox(self,fyear,page,tbno,seq,tbox,pp):
        super().add_textbox(fyear,page,tbno,seq,tbox)
        cdate = tbox['date']
        rate = tbox['rate']
        fyear = tbox['fyear']
        earnings = tbox['earnings']
        tbx = self.textbox[fyear][page][tbno][seq]
        self.decode_earnings(fyear,cdate,rate,earnings,tbx,pp)
        
    def get_textboxes(self):
        return(super().get_textboxes())
        
    def get_name(self):
        return(super().get_name())
    
    def get_cba_salaries(self):
        return(self.cba)
    
    def get_uuid(self):
        return(super().get_uuid())
    
    def decode_earnings(self,fyear,cdate,rate,earnings,tbx,pp):          #compute step
        sy = pp.get_school_year(cdate)
        cbix = int(sy[:4]) - 2013
        maxdiff = 99.9
        step = np.NaN
        for i in np.arange(1,7):
            if (abs(self.cba[cbix,i] - rate) < maxdiff):
                maxdiff = abs(self.cba[cbix,i] - rate)
                step = i + 1
        if (maxdiff < 0.005):
            step_code = sy[:4] + '-' + str(step)
        else:
            step_code = ''
        tbx['step'] = step_code
        return
    
class Paraprofessional_Substitute(Person):
    
    def __init__(self,name,uuid):
        super().__init__(name,uuid)
        return
        
    def add_payperiod(self,date):
        return(super().add_payperiod(date))
    
    def add_textbox(self,fyear,page,tbno,seq,tbox):
        return(super().add_textbox(fyear,page,tbno,seq,tbox))
        
    def get_textboxes(self):
        return(super().get_textboxes())
        
    def get_name(self):
        return(super().get_name())
    
    def get_uuid(self):
        return(super().get_uuid())
    
class Secretary_Clerk(Person):
    
    def __init__(self,name,uuid):
        super().__init__(name,uuid)
        self.cba = np.zeros((10, 7))    #salary matrix is 2-D numpy array 
                                        #indexed by: fyear, step#start with FY2016 (2016-2017) salary matrix

        self.cba[3,:] = np.array([[16.76, 17.26, 17.97, 18.99, 19.97, 20.96, 22.83]])   #FY2017
        
        self.cba[2,:] = np.around(self.cba[3,:]/1.02,2)
        self.cba[4,:] = self.cba[3,:]                                                   #FY2018  no change
        self.cba[5,:] = np.around(1.02*self.cba[4,:],2)                                 #FY2019  2% increase
        self.cba[6,:] = np.around(1.02*self.cba[5,:],2)                                 #FY2020  2% increase
        self.cba[7,:] = self.cba[6,:]                                                   #FY2021  no change?
        self.cba[8,:] = self.cba[7,:]                                                   #FY2022  no change?
        self.cba[9,:] = self.cba[8,:]                                                   #FY2023  no change?
        return
        
    def add_payperiod(self,date):
        return(super().add_payperiod(date))
    
    def add_textbox(self,fyear,page,tbno,seq,tbox,pp):
        super().add_textbox(fyear,page,tbno,seq,tbox)
        cdate = tbox['date']
        rate = tbox['rate']
        fyear = tbox['fyear']
        earnings = tbox['earnings']
        tbx = self.textbox[fyear][page][tbno][seq]
        self.decode_earnings(fyear,cdate,rate,earnings,tbx,pp)
        
    def get_textboxes(self):
        return(super().get_textboxes())
        
    def get_name(self):
        return(super().get_name())
    
    def get_uuid(self):
        return(super().get_uuid())
    
    def decode_earnings(self,fyear,cdate,rate,earnings,tbx,pp):          #compute step
        sy = pp.get_school_year(cdate)
        cbix = int(sy[:4]) - 2013
        maxdiff = 99.9
        step = np.NaN
        for i in np.arange(1,7):
            if (abs(self.cba[cbix,i] - rate) < maxdiff):
                maxdiff = abs(self.cba[cbix,i] - rate)
                step = i + 1
        if (maxdiff < 0.005):
            step_code = sy[:4] + '-' + str(step)
        else:
            step_code = ''
        tbx['step'] = step_code
        return

class Central_Office_Secretary(Person):
    
    def __init__(self,name,uuid):
        super().__init__(name,uuid)
        self.cba = np.zeros((10, 7))    #salary matrix is 2-D numpy array 
                                        #indexed by: fyear, step#start with FY2016 (2016-2017) salary matrix

        self.cba[3,:] = np.array([[20.43, 20.93, 21.64, 22.65, 23.62, 24.63, 26.60]])   #FY2017
        
        self.cba[2,:] = np.around(self.cba[3,:]/1.02,2)
        self.cba[4,:] = self.cba[3,:]                                                   #FY2018  no change
        self.cba[5,:] = np.around(1.02*self.cba[4,:],2)                                 #FY2019  2% increase
        self.cba[6,:] = np.around(1.02*self.cba[5,:],2)                                 #FY2020  2% increase
        self.cba[7,:] = self.cba[6,:]                                                   #FY2021  no change?
        self.cba[8,:] = self.cba[7,:]                                                   #FY2022  no change?
        self.cba[9,:] = self.cba[8,:]                                                   #FY2023  no change?
        return
        
    def add_payperiod(self,date):
        return(super().add_payperiod(date))
    
    def add_textbox(self,fyear,page,tbno,seq,tbox,pp):
        super().add_textbox(fyear,page,tbno,seq,tbox)
        cdate = tbox['date']
        rate = tbox['rate']
        fyear = tbox['fyear']
        earnings = tbox['earnings']
        tbx = self.textbox[fyear][page][tbno][seq]
        self.decode_earnings(fyear,cdate,rate,earnings,tbx,pp)
        
    def get_textboxes(self):
        return(super().get_textboxes())
        
    def get_name(self):
        return(super().get_name())
    
    def get_uuid(self):
        return(super().get_uuid())
    
    def decode_earnings(self,fyear,cdate,rate,earnings,tbx,pp):          #compute step
        sy = pp.get_school_year(cdate)
        cbix = int(sy[:4]) - 2013
        maxdiff = 99.9
        step = np.NaN
        for i in np.arange(1,7):
            if (abs(self.cba[cbix,i] - rate) < maxdiff):
                maxdiff = abs(self.cba[cbix,i] - rate)
                step = i + 1
        if (maxdiff < 0.005):
            step_code = sy[:4] + '-' + str(step)
        else:
            step_code = ''
        tbx['step'] = step_code
        return

class Technician(Person):
    
    def __init__(self,name,uuid):
        super().__init__(name,uuid)
        return
        
    def add_payperiod(self,date):
        return(super().add_payperiod(date))
    
    def add_textbox(self,fyear,page,tbno,seq,tbox):
        return(super().add_textbox(fyear,page,tbno,seq,tbox))
        
    def get_textboxes(self):
        return(super().get_textboxes())
        
    def get_name(self):
        return(super().get_name())
    
    def get_uuid(self):
        return(super().get_uuid())

class Custodian(Person):
    
    def __init__(self,name,uuid):
        super().__init__(name,uuid)
        return
        
    def add_payperiod(self,date):
        return(super().add_payperiod(date))
    
    def add_textbox(self,fyear,page,tbno,seq,tbox):
        return(super().add_textbox(fyear,page,tbno,seq,tbox))
        
    def get_textboxes(self):
        return(super().get_textboxes())
        
    def get_name(self):
        return(super().get_name())
    
    def get_uuid(self):
        return(super().get_uuid())
    

class RIDE_Obj_labels():
    """Provides a dictionary of RIDE Obj descriptions"""
    
    def __init__(self):                                        #download RIDE UCOA data and extract Obj codes
            
        self.obj_labels = {}

        fydf = pd.read_csv("../Obj.csv") #read the file for this fiscal year
        
        objdd = fydf.to_dict()

        for key in objdd['obj'].keys():
            obj = int(str(objdd['obj'][key]))
            self.obj_labels[obj] = objdd['obj_desc'][key]
            
    def get_obj_desc(self,obj):                                            #look up RIDE Obj description
        """Provides a lookup function for Obj description"""
        try:
            obj_desc = self.obj_labels[obj]                                #return description if found
        except KeyError:
            obj_desc = '(none)'                                            #otherwise return '(none)'
                
        return(obj_desc)

class payperiod():                                      #class for dates at end of payperiods
    """Provides a dictionary of payroll periods"""
    
    def __init__(self):
    
        self.payperiods = {}                            #initialize payperiods dictionary

        delta = timedelta(days=14)                      #14 days per pay period
        first_payperiod = date(2009,7,3)                #first pay period of FY2010
        cutoff_date = date(2026,7,1)
        
        current_payperiod = first_payperiod
        current_year = first_payperiod.year
        current_month = first_payperiod.month

        while (current_payperiod < cutoff_date):         #generate pay periods through cutoff date
            self.payperiods[current_payperiod] = {}     #create empty dictionary for this payperiod
            fyear = current_payperiod.year              #compute fiscal year
            if (current_payperiod.month >= 7):          #for months7-12 it's current year plus 1
               fyear += 1
            self.payperiods[current_payperiod]['fyear'] = fyear
            schyear = current_payperiod.year
            if (current_payperiod.month < 8):
               schyear = str(current_payperiod.year-1) + '-' + str(current_payperiod.year)
            elif (current_payperiod.month > 8):
               schyear = str(current_payperiod.year) + '-' + str(current_payperiod.year+1)
            elif (current_payperiod.day < 14):
               schyear = str(current_payperiod.year-1) + '-' + str(current_payperiod.year)
            else:
               schyear = str(current_payperiod.year) + '-' + str(current_payperiod.year+1)
            self.payperiods[current_payperiod]['school_year'] = schyear
            self.payperiods[current_payperiod]['calendar_year'] = current_payperiod.year
            if ((current_payperiod.month == 6) & \
                ((current_payperiod+delta).month==7) & \
                ((current_payperiod+delta).day > 1)):
                self.payperiods[current_payperiod]['spans_fyear'] = 'True'
            else:
                self.payperiods[current_payperiod]['spans_fyear'] = 'False'
            current_payperiod += delta                                  #increment date by 14 days
        
        self.payperiods[date(2016,11,10)] = self.payperiods.pop(date(2016,11,11))
        return
    
    def get_payperiods(self):
        return(self.payperiods)
    
    def get_payperiod_end(self,fyear,ppno):                              #look up the date of the nth pay period
        """Lookup end date of payroll periods in a given fiscal year"""
        try:
            ppend = self.payperiods[fyear][ppno]
        except KeyError:
            ppend = np.NaN
            
        return(ppend)    
        
    def get_fiscal_year(self,xdate):                    #look up the fiscal year given a date
        """Lookup fiscal year given date"""
        fyr = xdate.year
        mon = xdate.month
        if (mon > 6):
            fyr += 1
        return(fyr)
    
    def get_school_year(self,xdate):                    #look up the fiscal year given a date
        """Lookup school year given date"""
        return(self.payperiods[xdate]['school_year'])
    
    def get_payperiod_no(self,fyear,xdate):             #look up the pay period number for a date
        """Lookup payroll period given a date"""
        period=1
        while self.payperiods[fyear][period] >= xdate:
            period += 1
        return(period)
    
    def get_next_payday(self,y,m,d):                                    #find the next payday after given date
        """Find the end of the current pay period given a date"""
        d = date(y,m,d)                                                 #convert y,m,d to date value
    
        for pdate in self.payperiods.keys():                            #look through paydates
            if (payday >= d):                                       #return the first one greater than 
                return(payday)                                      #the date supplied
            
        return(np.NaN)
    
    def get_previous_payday(self,cdate):                            #find the previous payday
        """Find the date of the previous payday"""
        
        tdate = np.NaN
        
        for pdate in sorted(self.payperiods.keys()):                #look through paydates
            if (pdate <= cdate):                                    #return the last one less than or equal
                tdate = pdate
            
        return(tdate)
    

### Instantiate classes

Instantiates classes that provide various functions

In [ ]:
from datetime import *
from datascience import *

In [ ]:
ac = EG_acct_codes()    #EG accounting codes class
print('get_eg_acct_desc(acct): ',ac.get_eg_acct_desc.__doc__)

od = RIDE_Obj_labels()    #lookup function for RIDE Obj descriptions
print('get_obj_desc(obj): ',od.get_obj_desc.__doc__)

pp = payperiod()          #get end dates of payroll periods
print('get_payperiod_end(fyear,ppno): ',pp.get_payperiod_end.__doc__)

### Create people objects and store textboxes

In [ ]:
import uuid

paydays = pp.get_payperiods()

teachers = {}
teacher_substitutes = {}
paras = {}
para_substitutes = {}
secretaries = {}
co_secretaries = {}
technicians = {}
custodians = {}
persons = {}

ay = {}                                                           #combine FY17 and FY18
ay[2017] = consolidated_names17
ay[2018] = consolidated_names18
                                                                  #build people dictionary for all years 
for year in ay.keys():                                            # loop through years
    for page in ay[year].keys():                                  # loop through pages of earnings report
        for tb in ay[year][page].keys():                          # loop through text boxes on each page
            fund = ay[year][page][tb]['fund']
            acct = ay[year][page][tb]['acct']
            obj  = ay[year][page][tb]['obj']
            
            txt  = ay[year][page][tb]['text']                      # pick up names
            dates = ay[year][page][tb]['dates']
            numbers = ay[year][page][tb]['numbers']
            positions = ay[year][page][tb]['positions']
            rates     = ay[year][page][tb]['rates']
            earnings  = ay[year][page][tb]['earnings']
            
            lines  = txt.split('\n')                               # split names string into lines
            plines = positions.split('\n')
            rlines = rates.split('\n')
            elines = earnings.split('\n')
            dlines = dates.split('\n')
            nlines = numbers.split('\n')
            
            for i in np.arange(len(lines)):                       # add name to people dictionary
                if (len(lines[i]) > 1):
                    name = lines[i]
                    datestr = dlines[i]
                    dcomp = datestr.split('/')
                    check_date = date(int(dcomp[2]),int(dcomp[0]),int(dcomp[1]))
                    if (check_date not in paydays.keys()):
                        new_date = pp.get_previous_payday(check_date)
                        print('adjusting date',name,check_date,new_date)
                        check_date = new_date
                    try:
                        check = nlines[i]
                    except IndexError:
                        check = ''
                    position = plines[i]
                    tbinfo = {}
                    tbinfo['fyear'] = year
                    tbinfo['page'] = page
                    tbinfo['textbox'] = tb
                    tbinfo['seq'] = i
                    tbinfo['fund'] = fund
                    tbinfo['acct'] = acct
                    tbinfo['obj'] = obj
                    tbinfo['date'] = check_date
                    tbinfo['check'] = check
                    tbinfo['position'] = position
                    tbinfo['rate'] = float(rlines[i])
                    tbinfo['earnings'] = round(float(elines[i]),2)
                    tbinfo['acct_desc'] = ac.get_eg_acct_desc(acct)
                    tbinfo['obj_desc'] = od.get_obj_desc(float(obj))

                    if ('TEACHER' in position):
                        if (name not in teachers.keys()):
                            uuidOne = uuid.uuid1()
                            teachers[name] = Teacher(name,uuidOne)
                        teachers[name].add_textbox(year,page,tb,i,tbinfo)
                    elif ('T/S' in position):
                        if (name not in teacher_substitutes.keys()):
                            uuidOne = uuid.uuid1()
                            teacher_substitutes[name] = Teacher_Substitute(name,uuidOne)
                        teacher_substitutes[name].add_textbox(year,page,tb,i,tbinfo)
                    elif (('PARAPROFES' in position) | ('SPED PARA' in position)):
                        if (name not in paras.keys()):
                            uuidOne = uuid.uuid1()
                            paras[name] = Paraprofessional(name,uuidOne)
                        paras[name].add_textbox(year,page,tb,i,tbinfo,pp)
                    elif ('P/S' in position):
                        if (name not in para_substitutes.keys()):
                            uuidOne = uuid.uuid1()
                            para_substitutes[name] = Paraprofessional_Substitute(name,uuidOne)
                        para_substitutes[name].add_textbox(year,page,tb,i,tbinfo)
                    elif ('SECRETA' in position):
                        if (acct == '71419901'):
                            if (name not in co_secretaries.keys()):
                                uuidOne = uuid.uuid1()
                            co_secretaries[name] = Central_Office_Secretary(name,uuidOne)
                            co_secretaries[name].add_textbox(year,page,tb,i,tbinfo,pp)
                        else:
                            if (name not in secretaries.keys()):
                                uuidOne = uuid.uuid1()
                            secretaries[name] = Secretary_Clerk(name,uuidOne)
                            secretaries[name].add_textbox(year,page,tb,i,tbinfo,pp)
                    elif ('SUPPORT SP' in position):
                        if (name not in technicians.keys()):
                            uuidOne = uuid.uuid1()
                            technicians[name] = Technician(name,uuidOne)
                        technicians[name].add_textbox(year,page,tb,i,tbinfo)
                    elif ('CUSTODIAN' in position):
                        if (name not in custodians.keys()):
                            uuidOne = uuid.uuid1()
                            custodians[name] = Custodian(name,uuidOne)
                        custodians[name].add_textbox(year,page,tb,i,tbinfo)
                    else:
                        if (name not in persons.keys()):
                            uuidOne = uuid.uuid1()
                            persons[name] = Person(name,uuidOne)
                        persons[name].add_textbox(year,page,tb,i,tbinfo)

In [ ]:
teachers['xxxxx'].get_textboxes()

In [ ]:
for name in teachers.keys():
    uuid = teachers[name].get_uuid()
    print("\nName: ",name,'\n')
    tbxs = teachers[name].get_textboxes()
    for fyear in tbxs.keys():
        sseq = 1
        for page in tbxs[fyear].keys():
            for tb in tbxs[fyear][page].keys():
                for seq in tbxs[fyear][page][tb].keys():
                    tbx = tbxs[fyear][page][tb][seq]
                    earnings = tbx['earnings']
                    acct = tbx['acct']
                    #cdate =tbx['date']
                    ciso = tbx['date'].isoformat()
                    acct_desc = tbx['acct_desc']
                    try:
                        school_year = tbx['school_year']
                    except KeyError:
                        school_year = ' '
                    obj = tbx['obj']
                    if (1==1):
                        obj_desc = tbx['obj_desc']
                        try:
                            step = tbx['step']
                        except KeyError:
                            step = ' '
                        try:
                            fte = tbx['fte']
                        except KeyError:
                            fte = ' '
                        rate = tbx['rate']
                        if ((float(rate) > 150.) & (earnings > 50.)):
                            try:
                                payments = tbx['payments']
                            except KeyError:
                                payments = ' '
                            mindiff = tbx['mindiff']
                            if (mindiff > 25.):
                                payments = '    '
                                fte = '   '
                            print("    ",fyear,ciso,school_year,acct,acct_desc,obj,\
                                  obj_desc,step,fte,payments,rate,earnings,mindiff)
                            sseq += 1

In [ ]:
co_secretaries['xxxx'].get_textboxes()

In [ ]:
for name in teachers.keys():
    tbxs = teachers[name].get_textboxes()
    for fyear in tbxs.keys():
        for page in tbxs[fyear].keys():
            for tb in tbxs[fyear][page].keys():
                for seq in tbxs[fyear][page][tb].keys():
                    earnings = tbxs[fyear][page][tb][seq]['earnings']
                    acct = tbxs[fyear][page][tb][seq]['acct']
                    acct_desc = tbxs[fyear][page][tb][seq]['acct_desc']
                    obj = tbxs[fyear][page][tb][seq]['obj']
                    if ('51110' in obj):
                        obj_desc = tbxs[fyear][page][tb][seq]['obj_desc']
                        try:
                            step = tbxs[fyear][page][tb][seq]['step']
                        except KeyError:
                            step = ' '
                            print(name, fyear,tbxs[fyear][page][tb][seq])
                        try:
                            fte = tbxs[fyear][page][tb][seq]['fte']
                        except KeyError:
                            fte = ' '
                        rate = tbxs[fyear][page][tb][seq]['rate']
                        if (float(rate) > 150.):
                            try:
                                payments = tbxs[fyear][page][tb][seq]['payments']
                            except KeyError:
                                payments = ' '
                                

### Get totals by acct and fiscal year

In [ ]:
dct = {}
            
for name in teachers.keys():
    uuid = teachers[name].get_uuid()
    tbxs = teachers[name].get_textboxes()
    for fyear in tbxs.keys():
        if (fyear not in dct.keys()):
            dct[fyear] = {}
        for page in tbxs[fyear].keys():
            for tb in tbxs[fyear][page].keys():
                for seq in tbxs[fyear][page][tb].keys():
                    tbx = tbxs[fyear][page][tb][seq]
                    obj = tbx['obj']
                    if (obj == '51110'):
                        acct = tbx['acct']
                        if (acct not in dct[fyear].keys()):
                            dct[fyear][acct] = 0.0
                        earnings = tbx['earnings']
                        dct[fyear][acct] += earnings
                        
for fyear in dct.keys():
    print(fyear)
    for acct in dct[fyear].keys():
        desc = ac.get_eg_acct_desc(acct)
        print('     ',acct,desc,dct[fyear][acct])

In [ ]:
technicians

In [ ]:
paras

In [ ]:
pmts = {}

for name in teachers.keys():
    uuid = teachers[name].get_name()
    tb = teachers[name].get_textboxes()
    if (uuid not in pmts.keys()):
        pmts[uuid] = {}
    for fyear in tb.keys():
        if (fyear not in pmts[uuid].keys()):
            pmts[uuid][fyear] = {}
            
print("Distinct teacher names: ",len(pmts))
            
pmts
        

In [ ]:
for name in teachers.keys():
    uuid = teachers[name].get_uuid()
    #print("\nName: ",name,'\n')
    if ('MALLILO' in name):
        tbxs = teachers[name].get_textboxes()
        for fyear in tbxs.keys():
            if (fyear == 2017):
                sseq = 1
                for page in tbxs[fyear].keys():
                    for tb in tbxs[fyear][page].keys():
                        for seq in tbxs[fyear][page][tb].keys():
                            print(tbxs[fyear][page][tb][seq])

In [ ]:
unkacct = {}

for name in teachers.keys():
    uuid = teachers[name].get_uuid()
    #print("\nName: ",name,'\n')
    tbxs = teachers[name].get_textboxes()
    for fyear in tbxs.keys():
        sseq = 1
        for page in tbxs[fyear].keys():
            for tb in tbxs[fyear][page].keys():
                for seq in tbxs[fyear][page][tb].keys():
                    if ('(no description)'==tbxs[fyear][page][tb][seq]['acct_desc']):
                        acct = tbxs[fyear][page][tb][seq]['acct']
                        if (acct not in unkacct.keys()):
                            unkacct[acct] = {}
                        obj = tbxs[fyear][page][tb][seq]['obj']
                        if (obj not in unkacct[acct].keys()):
                            unkacct[acct][obj] = 0
                        unkacct[acct][obj] += 1
                        
unkacct
                    